## Overview

In this notebook we introduce the Kalman filter through consideration of the recursive least squares problem. This is a pretty deep dive into the mathematics at an early stage. I expect the approach will lose some students, but engage others. I'll follow on with a more intutive look at the the filter in a second lecture. The material presented is taken from:

"Computational Science and Engineering" by Gilbert Strang, Cambridge Press (2007)

A very intuitive overview of the topic that includes very clear statements of the formula is here:

https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/

### Motivating problem: Updating Averages

Suppose we have computed the average $\hat{u}_{99}$ of 99 numbers $b_1...b_{99}$. A new number $b_{100}$ arrives. How can we find the new average $\hat{u}_{100}$ the 100 $b$s, *without adding the first 99 all over again*? We want to use only $\hat{u}_{99}$ and $b_{100}$.

#### Solution

Think of the new average as a *weighted* average of the original average, $\hat{u}_{99}$ and the new observation, $b_{100}$. What are the weights? Those should be proportionate to the number of observations, shouldn't they? 

$$\hat{u}_{100} = \frac{99}{100} \hat{u}_{99} + \frac{1}{100}b_{100}$$

OK - That should make sense. Let me rearrange the terms to make a point.

$$\hat{u}_{100} = \hat{u}_{99} + \frac{1}{100}( b_{100} - \hat{u}_{99})$$

This is more useful because the terms can be identified in a different way.

* $\hat{u}_{100}$ This is something that is updated based on new knowledge, or observation.
* $\hat{u}_{99}$ This is the old thing that is being updated with new observation.
* $\frac{1}{100}$ This is a factor that lets us know how important the new observation is. This is called the ***gain factor***.
* $( b_{100} - \hat{u}_{99})$ This is how surprising the new thing is. It is called the ***innovation***.

The terms and the formula are simple in this case. Try and master them now. In what's to come we will address much more general situations, but the basics demonstrated here will apply.

### Recursive Least Squares
The case of adjusting the average was accessible and instructive. However, it is of limited use. Let's consider a different problem that has more importance. Suppose you have a set of parameters describing a curve. Those parameters were found by solving a ***least squares*** problem. I'll show how to do that in a moment, but the parameters minimize the misfit between a curve and some data. 

Now, suppose a new piece of data arrives. How does this new information/data alter the set of parameters? Further, can we devise a way of changing parameters to acknowledge new data that does not require us to redo the entire least squares problem? 

Just like above, where we avoided recomputing the average, we'd like to update least squares parameters without redoing the least squares problem. The approach follows.

#### Least squares primer
Suppose you've fit your data using least squares. As a result have a vector of parameters, $\mathbf{u}$ that represent the coefficients of a curve. You found those parameter by solving a linear algebra problem written as

$$\mathbf{A} \mathbf{u} = \mathbf{b}$$.

Let me give you a specific case to help you think about this. You measured your class mates' wrist circumferences, and heights because you think that wrist circumference is a good proxy for height. The matrix $\mathbf{A}$ represents the ***design matrix***, in each row are the observations that have been made, in this case wrist circumference. There might also be entries of 1 in a row, that represents an additive constant. Specifically, for this problem

$$\mathbf{A} = \begin{bmatrix} w_1 & 1\\
                               w_2 & 1\\
                               \vdots & \vdots\\
                               w_n & 1
                               \end{bmatrix} .$$
                               
Where the $w_i$ are the wrist measurements for $n$ classmates. Now, we're doing a linear fit, so $\mathbf{u}= (u_0,u_1)^T$ where $u_0$ is slope and $u_1$ is the intercept. I hope that now it is clear why the column of ones was needed in $\mathbf{A}$, it adds the $u_1$ to each equation. Finally, $\mathbf{b} = (b_0,b_1,..b_n)$ are the heights of each class member. Hence, the $i^{th}$ row of the linear algebra problem looks like

$$w_i u_0 + u_1 = b_i.$$


Solving this system is a little bit of a challenge because $\mathbf{A}$ isn't square. It's $n\times2$ ($n$ rows, $2$ columns). To solve, we end up considering $\mathbf{A}^T\mathbf{A}$, which is $n \times n$. These are the ***normal equation***. Specifically, to solve for the parameters we write.

$\mathbf{u} = \frac{\mathbf{A}^T}{\mathbf{A}^T\mathbf{A}}\cdot \mathbf{b}$.

Note $\frac{\mathbf{A}^T}{\mathbf{A}^T\mathbf{A}}$ can be determined from the *Moore-Penrose pseudoinverse*. This pseudoinverse operation is supported in most mathematical software packages. I believe we will return to the least squares problem later in the course. In Python, one would do

`
from numpy import array,dot
from numpy.linalg import pinv
A = array([[1,0,0],[-1,1,-0],[0,1,0]])
dot(pinv(dot(A.T,A)),A.T)
`

Which gives 

$$\frac{1}{3}\begin{bmatrix}2&-1 &1\\ 1&1&2\\0&0&0\end{bmatrix}$$
#### Recursive least squares

This lets us start addressing the problem we are really after. Supposing we have solved for $\mathbf{u}$, and then another person comes into our class. How can we include the data from that person in our estimate of $\mathbf{u}$. Even more importantly, can we include him in such a way that we avoid solving the entire normal equation again? Begin by writing the original problem with the new information included.

$$\begin{bmatrix} \mathbf{A_{old}} \\ \mathbf{A_{new}} \end{bmatrix}\cdot \mathbf{u_{new}} =
  \begin{bmatrix} \mathbf{b_{old}} \\ \mathbf{b_{new}} \end{bmatrix}$$
  
Solving this would get us the new parameters, $\mathbf{u_{new}}$, but that's the big job we're trying to avoid. Given that the updated $\mathbf{A}$ is $[\mathbf{A_{old}} \mathbf{A_{new}}]^T$, and that $\mathbf{A}^T\mathbf{A}$ is the key piece of the normal equations, try writing

$$\mathbf{A}^T\mathbf{A} = \mathbf{A_{old}}^T\mathbf{A_{old}} + \mathbf{A_{new}}^T\mathbf{A_{new}}$$

Similarly, the right hand side of the normal equation involves $\mathbf{A}^T\cdot\mathbf{b}$, which can be broken out in a similar way

$$\mathbf{A}^T\cdot\mathbf{b} = \mathbf{A_{old}}^T\cdot\mathbf{b_{old}} + \mathbf{A_{new}}^T\cdot\mathbf{b_{new}} = \mathbf{A_{old}}^T \mathbf{A_{old}}\mathbf{u_{old}} + \mathbf{A_{new}}^T\cdot\mathbf{b_{new}}$$

Where the original linear system, $\mathbf{A_{old}}\mathbf{u_{old}} = \mathbf{b_{old}}$ was used to replace $\mathbf{b_{old}}$. This relation can be modified by 1) replacing $\mathbf{A_{old}}^T\mathbf{A_{old}} = \mathbf{A}^T\mathbf{A} - \mathbf{A_{new}}^T\mathbf{A_{new}}$, and 2) multiplying through by $(\mathbf{A}^T\mathbf{A})^{-1}$. This gives:

$$(\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\cdot\mathbf{b} = (\mathbf{A}^T\mathbf{A})^{-1}(\mathbf{A}^T\mathbf{A} - \mathbf{A_{new}}^T\mathbf{A_{new}})\mathbf{u_{old}} + (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A_{new}}^T\cdot\mathbf{b_{new}}$$

The left hand side of this is the solution of the normal equation for the full system, so that is $\mathbf{u_{new}}$. Awesome, that's what we want. Distribution of the $(\mathbf{A}^T\mathbf{A})^{-1}$ on the right hand side, and rearranging terms yields

$$\mathbf{u_{new}} = \mathbf{u_{old}} + (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A_{new}}^T(\mathbf{b_{new}} - \mathbf{A_{new}}\cdot\mathbf{u_{old}}).$$

It took a little work, but now look at the perfect analogy to the problem of computing a new average.
* The ***innovation*** is now $\mathbf{b_{new}} - \mathbf{A_{new}}\cdot\mathbf{u_{old}}$
* The ***gain matrix*** is $\mathbf{K} = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A_{new}}^T$
* Otherwise, the update is in terms of the old solution with a additive correction.

Think carefully about how to compute each term and its meaning.

### Example
Finding an average can be written as a least squares problem, and provides a link between recursive least squares and the problem of updating averages. If $u_{99} = \frac{1}{99}(b_1 + ...b_{99})$ is actually a solution to a least squares problem, the $\mathbf{A_{old}} = [1 ... 1]^T$ and the problem is written

$$\begin{bmatrix}1\\ \vdots \\ 1\end{bmatrix} u_{99} =\begin{bmatrix}b_1\\ \vdots \\ b_{99}\end{bmatrix} $$

and the solution is very easily found by pre-multiplying both sides by $\mathbf{A_{old}}^T = [1 \ldots 1]$

$$[1 \ldots 1] \begin{bmatrix}1\\ \vdots \\ 1\end{bmatrix} u_{99} =[1 \ldots 1] \begin{bmatrix}b_1\\ \vdots \\ b_{99}\end{bmatrix} $$

$$99 \hat{u}_{99} = b_1 + \ldots + b_{99}$$

The parameter is found to be the average, as expected. Now consider a $b_{100}$, which we have been calling $b_{new} = 100$. Use the previously found formulas to update the average. We'll need $\mathbf{A_{new}}$, which should just be $[1]$ because the 100$^{th}$ equation is just $u_{100} = b_{100} = b_{new}$. Additionally, we need $\mathbf{A}^T\mathbf{A}$ which is 99 (old) + 1 (new) = 100.

Applying what we know in the last equation of the previous section:

$$\mathbf{u_{new}} = \mathbf{u_{old}} + (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A_{new}}^T(\mathbf{b_{new}} - \mathbf{A_{new}}\cdot\mathbf{u_{old}}).$$


$$\hat{u}_{100} = \hat{u}_{99} + (100)^{-1}(b_{100} - u_{99}).$$

Which matches our introductory result.



### Kalman Filter by Example
Now imagine that the parameters change in time - $\mathbf{u}$ becomes $\mathbf{u_i}$, where the $i$ refers to some discrete time step. Further, assume that rather than parameters, these are the *state* of a system - so things like the velocity and position of a moving airplane. This is the Kalman filter. The state is determined by measurements, or $\mathbf{b_i}$ (again, $i$ is the discrete time). The previous times $\mathbf{b_{0 \ldots i-1}}$ matter to the state $\mathbf{u_i}$, but not as much as the current measurement, $\mathbf{b_i}$.

#### Example: Heart Rate
Suppose $u$ is the only state variable. It represents your heart rate. It is measured twice, $b_1$ and some time later $b_2$. Suppose $b_2$ is so much later that its difference from $b_1$ could reflect aging processes. Years, not days. This means that we have a *state equation*

$$u_2 - u_1 = c + \textrm{error } \epsilon_1$$

Where $c$ is a number that reflects the natural rate of decrease as you age. Remember, $u$ is the estimate, $b$ is the measurement. They are not quite the same, because we also have the model. *The purpose will be to reconcile measurements with model.* The following system of equations must be solved to reconcile model with measurement:

$$\begin{bmatrix}1 & 0 \\ -1 & 1 \\ 0 & 1\end{bmatrix} \begin{bmatrix} u_1\\u_2\end{bmatrix} = \begin{bmatrix}b_1\\c\\b_2\end{bmatrix}$$

This is also understandable in terms of recursive least squares. The problem is equivalent to

$$\begin{bmatrix}\mathbf{A_{old}} \\ \mathbf{A_{state}} \\ \mathbf{A_{new}} \end{bmatrix} \begin{bmatrix} u_{old}\\u_{new} \end{bmatrix} = \begin{bmatrix}b_{old}\\c_{state}\\b_{new}\end{bmatrix}$$

##### Errors
I've been a casual with the errors so far, so as not to bog down discussion. However, they play a critical role in any Kalman filter. Think about the least squares problem being solved here. Effectively, we are minimizing the following when we solve the problem above

$$\frac{1}{\sigma_1^2}(b_1-u_1)^2 + \frac{1}{v_1^2}(c + u_1 - u_2)^2 + \frac{1}{\sigma_2^2}(b_2-u_2)^2.$$

Where $\sigma_1^2$ and $\sigma_2^2$ are errors in the measurements $b_i$ and $v_1^2$ is the error in the state. To account for the errors in the linear system, we write

$$\mathbf{A}^T\mathbf{C}\mathbf{A} \mathbf{u} = \mathbf{A}^T\mathbf{C}\mathbf{b}$$

where $\mathbf{C}^{-1} = \textrm{diag}[\sigma_1^2, v_1^2, \sigma_2^2]$.

##### Solution
For simplicity, assume that $\mathbf{C}=\mathbf{I}$ (errors are all 1), then the normal equation $\mathbf{A}^T\mathbf{A} \mathbf{u} = \mathbf{A}^T\mathbf{b}$, becomes

$$ \begin{bmatrix} 2 & -1\\ -1 & 2 \end{bmatrix} \begin{bmatrix} u_1\\u_2 \end{bmatrix} = \begin{bmatrix}b_1-c\\b_2 + c\end{bmatrix}$$

The solution of this system is instructional to write

$$u_1 = \frac{1}{3}(2b_1+b_2-c)$$
$$u_2 = \frac{1}{3}(b_1+2b_2+c)$$

As expected, we see that the observations are given greater weight to their respective estimates, $b_1$ contributes more to $u_1$ and $b_2$ contributes more to $u_2$. Both take into account the 'model' of a slowing heart rate, $c$. 

This is called Kalman *smoothing* because the calculation of $u_1$ is aware of the observation $b_1$. This is a good qay of doing things when all the observations are available.


## Conclusion
The above discussion is deep. As it concludes, you might think that Kalman smoothing is the norm, and that you are solving large systems that grow with the number of observations. This is often not the case. More common is the situation where you model where something will be, based on a GPS based velocity, and then measure where it is. See homework problem. You then reconcile the two. 

Let me introduce terminology to make that clear. Suppose $\hat{u}_{i|j}$ is read: the estimate of $u$ at time $i$ using information through time $j$, where $j \le i$. With this notation, we would say that the most common situation is that we have $\hat{u}_{i-1|i-1}$ and we seek to get $\hat{u}_{i|i}$. The linear algebra problem to be solved is just like recursive least squares. The solutions are summarized below. This is the main take away from this lecture, these formula will get you what you want with a Kalman filter - most of the time.

#### Prediction
$$ \mathbf{\hat{u}_{i|i-1}} = \mathbf{F_{i-1}} \mathbf{\hat{u}_{i-1|i-1}} + \mathbf{c_i}$$

Note this is application of the *model only*. Here $\mathbf{F_{i-1}}$ is a matrix that transforms observations, like position to velocity. In the trival heart rate example, it is just 1. In the homework, it'll be something more complex. $\mathbf{c_i}$ is some constant, for example, in the recent problem the rate of heart slowing with age.

#### Correction
$$ \mathbf{\hat{u}_{i|i}} = \mathbf{\hat{u}_{i|i-1}} + \mathbf{K_i} (\mathbf{b_i} - \mathbf{A_i} \mathbf{\hat{u}_{i|i-1}})$$

Review recursive least squares above and note the similarity in the correction forms. All that is new here is that we have a model to get the first estimate $\mathbf{\hat{u}_{i|i-1}}$. Also, recall that the so-called gain term remains $\mathbf{K_i=(A^TA)^{-1}A^T_{new}}$, but consistent with previous discussion, could be $\mathbf{(A^TCA)^{-1}A^T_{new}}C$ if errors are taken into account. The relative magnitude of this gain is telling you how much the state is changing - big $\mathbf{K_i}$ the state is changing a lot - like a vehicle braking rapidly. Small $\mathbf{K_i}$, slow change, like a car that is just cruising along.


### Final Example: Kalman filter for heart rates
Return to the heart rate issue, but now compute the $\mathbf{K_i}$ recursively, as new information is acquired.

### First measurement
$$u_1 = b_1$$
Then $A_{1|1} = [1]$, and $(A^TA)^{-1}_{1|1} = [1]$.
### Add model estimate
$$u_2 - u_1 =  c_1$$
Even though we don't have $u_2$ yet, we can write this with $\mathbf{A_{2|1}} = \begin{bmatrix}1 & 0 \\ -1 & 1\end{bmatrix}$ and $\mathbf{(A^TA)^{-1}_{2|1}} = \begin{bmatrix} 1 & 1 \\ 1 & \mathbf{2} \end{bmatrix}$.
### Include  next measurement
$$ u_2 = b_2$$
This gives updates $\mathbf{A_{2|2}} = \begin{bmatrix}1 & 0 \\ -1 & 1\\0 & 1\end{bmatrix}$ and $\mathbf{(A^TA)^{-1}_{2|2}} =\frac{1}{3} \begin{bmatrix} 2 & 1 \\ 1 & \mathbf{2} \end{bmatrix}$.

### Working through the Kalman filter.
Based on the above, the first estimate is
$$\hat{u}_{1|1} = b_1$$

Next, we incorporate the model, using the 'predictor' equation above gives:
$$\hat{u}_{2|1} = \hat{u}_{1|1} + c_1 = b_1 + c_1$$

Now it gets more interesting because values for the $\mathbf{K_i}$ and $\mathbf{A_i}$ appearing in the equation for correction aren't obvious. What to use there? 

Short answer - $\mathbf{K_i}$ is the variance in variable $i$, which corresponds to the final entry in $(\mathbf{AA^T})^{-1}_{i|i}$. $\mathbf{A_i}$, likewise, corresponds to the final value in the $\mathbf{A_{i|i}}$ matrix!

$$\hat{u}_{2|2} = \hat{u}_{2|1} + \mathbf{K_2}(b_1-\mathbf{A_2} \hat{u}_{2|1}) = 
b_1 + c_1 + \frac{2}{3} (b_2- 1 (b_1 + c_1))$$

$$\hat{u}_{2|2} = \frac{1}{3} ( b_1 + 2b_2 +c_1)$$

## Worked Problems

### Heart rate with one more measurement
Extend the matrix $\mathbf{A}$ in the previous problem to be 5 $\times$ 3 with $u_3 - u_2 = c_2$ and a new measurement $u_3 = b_3$. With unit variances $\mathbf{C} = \mathbf{I}$, solve the normal equation for the best estimates $\hat{u}_1,\hat{u}_2,\hat{u}_3$. Note this is *smoothing* because the estimates of $\hat{u}_1,\hat{u}_2$ depend on the later information found in $\hat{u}_3$.

In [9]:
from pylab import *
# The rows of the matrix will be u1,u2-u1,u2,-u2+u3,u3
A = array([[1,0,0],[-1,1,0],[0,1,0],[0,-1,1],[0,0,1]])

dot(pinv(dot(A.T,A)),A.T)  # This is (A.T*A)**-1 * A.T
# The result of the above, dotted with b, will give each of the estimates of u, one per row.

array([[ 0.625, -0.375,  0.25 , -0.125,  0.125],
       [ 0.25 ,  0.25 ,  0.5  , -0.25 ,  0.25 ],
       [ 0.125,  0.125,  0.25 ,  0.375,  0.625]])

#### Predictor-Corrector formula
Continue the Kalman recursion from $\hat{u}_{2|2}$ above, to predict $\hat{u}_{3|2}$ and correct to be $\hat{u}_{3|3}$. Find their variances $P_{3|2}$ and $P_{3|3}$ from the last entries in $(\mathbf{AA^T})^{-1}_{3|2}$ and $(\mathbf{AA^T})^{-1}_{3|3}$.

Here, we first add a new prediction for state, based on the model
$$u_3-u_2 = c_2$$

This results in an updated matrix. Add one row for the equation of state, and one more column for the new variable, $u_3$
$$\mathbf{A_{3|2}} = \begin{bmatrix}1 & 0 & 0 \\ -1 & 1 & 0\\0 & 1 & 0\\0&-1&1\end{bmatrix}$$

and,
$$(\mathbf{A^TA})_{3|2}^{-1} = \frac{1}{3} \begin{bmatrix}2 & 1 & 1 \\ 1 & 2 & 2\\ 1 & 2 &5 \end{bmatrix}$$

Another measurement, $b_3$ gives the equation
$$u_3 = b_3$$

adding a row the $\mathbf{A}$ matrix for the new observation, or equation $u_3=b_3$ gives
$$\mathbf{A_{3|2}} = \begin{bmatrix}1 & 0 & 0 \\ -1 & 1 & 0\\0 & 1 & 0\\0&-1&1 \\ 0&0&1\end{bmatrix}$$.

With
$$(\mathbf{A^TA})_{3|3}^{-1} = \frac{1}{8} \begin{bmatrix}5 & 2 & 1 \\ 2 & 4 & 2\\ 1 & 2 & \mathbf{5} \end{bmatrix}$$

Now, we are ready to work. First the prediction is straight forward from the model,

$$\hat{u}_{3|2} = \hat{u}_{2|2} + c_2 = b_2 + c_2$$

The correction, is slightly more challenging, but understanding the gain is just the variance $P_{3|3}$ is most of the problem, this gives

$$\hat{u}_{3|3} = \mathbf{\hat{u}_{3|2}} + \mathbf{K_3} (\mathbf{b_i} - \mathbf{A_3} \mathbf{\hat{u}_{3|2}}) = b_2 + c_2 + \frac{5}{8} (b_3 - 1 ( b_2 + c_2)) $$

$$\hat{u}_{3|3} = \frac{1}{8} ( 5b_3 + 3b_2 + 3c_2)$$

To get this, I used some simple python to get the inverses of matrices. That is below.


In [4]:
from numpy import array,dot
from numpy.linalg import pinv
A = array([[1,0,0],[-1,1,-0],[0,1,0],[0,-1,1]])
print pinv(dot(A.T,A))
An = array([[1,0,0],[-1,1,-0],[0,1,0],[0,-1,1],[0,0,1]])
print pinv(dot(An.T,An))

[[0.66666667 0.33333333 0.33333333]
 [0.33333333 0.66666667 0.66666667]
 [0.33333333 0.66666667 1.66666667]]
[[0.625 0.25  0.125]
 [0.25  0.5   0.25 ]
 [0.125 0.25  0.625]]
